In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [5]:
from google.colab import drive
drive.mount('/content/drive')

data_dir ="/content/drive/MyDrive/Data_COVID"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
img_height,img_width=400, 400
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2480 files belonging to 2 classes.
Using 1984 files for training.


In [15]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2480 files belonging to 2 classes.
Using 496 files for validation.


In [16]:
class_names = train_ds.class_names
print(class_names)

['COVID', 'non-COVID']


In [23]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(400,400,3),
                   pooling='avg',classes=5,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(5, activation='softmax'))

In [24]:
resnet_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper_6 (ModuleWra  (None, 2048)             0         
 pper)                                                           
                                                                 
 module_wrapper_7 (ModuleWra  (None, 512)              1049088   
 pper)                                                           
                                                                 
 module_wrapper_8 (ModuleWra  (None, 5)                2565      
 pper)                                                           
                                                                 
Total params: 24,639,365
Trainable params: 1,051,653
Non-trainable params: 23,587,712
__________________________________

In [27]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [28]:
epochs=10
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Epoch 1/10
62/62 [==============================] - 915s 13s/step - loss: 0.6187 - accuracy: 0.7535 - val_loss: 0.3411 - val_accuracy: 0.8306
Epoch 2/10
62/62 [==============================] - 27s 416ms/step - loss: 0.2494 - accuracy: 0.8967 - val_loss: 0.2489 - val_accuracy: 0.8931
Epoch 3/10
62/62 [==============================] - 27s 417ms/step - loss: 0.2328 - accuracy: 0.9073 - val_loss: 0.3747 - val_accuracy: 0.8327
Epoch 4/10
62/62 [==============================] - 27s 413ms/step - loss: 0.2074 - accuracy: 0.9128 - val_loss: 0.2169 - val_accuracy: 0.9254
Epoch 5/10
62/62 [==============================] - 27s 415ms/step - loss: 0.1930 - accuracy: 0.9194 - val_loss: 0.3237 - val_accuracy: 0.8629
Epoch 6/10
62/62 [==============================] - 28s 434ms/step - loss: 0.1198 - accuracy: 0.9551 - val_loss: 0.1805 - val_accuracy: 0.9456
Epoch 7/10
62/62 [==============================] - 27s 412ms/step - loss: 0.0920 - accuracy: 0.9718 - val_loss: 0.1365 - val_accuracy: 0.9516


In [39]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",  # Specify that you're creating the test subset
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)


Found 2480 files belonging to 2 classes.
Using 496 files for validation.


In [45]:
test_loss, test_accuracy = resnet_model.evaluate(test_ds)
print("Test accuracy:", test_accuracy)


16/16 [==============================] - 8s 291ms/step - loss: 0.1250 - accuracy: 0.9597
Test accuracy: 0.9596773982048035
